In [ ]:
import os
from os import path
import networkx as nx
import ipycytoscape
import pandas as pd

BASEDIR = 'system'

IGNORE_ENVS = ["ignoreme"]
IGNORE_HOSTS = ["ignoreme"]

envs = os.listdir(path='system')
G=nx.Graph()
G.add_node(BASEDIR, label=BASEDIR, category='system')
system = list()

for env in envs:
    if env in IGNORE_ENVS:
        continue
    G.add_node(env, label=env, category="environment")
    G.add_edge(BASEDIR, env)
    hosts = os.listdir(path=path.join(BASEDIR, env))
    for host in hosts:
        if host in IGNORE_HOSTS:
            continue
        G.add_node(host, label=host, category="host")
        G.add_edge(env, host)
        processes = os.listdir(path=path.join(BASEDIR, env, host, 'bin'))
        for instance in processes:
            proc = f"{env}_{host}_{instance}"
            proctype = instance.rstrip("1234567890")
            G.add_node(proc, label=f"{instance}", category="instance")
            G.add_edge(host, proc)
            row = {'system': BASEDIR, 'environment': env, 'host': host, 'process': proctype, 'instance': instance}
            system.append(row)


In [ ]:
df = pd.DataFrame(system)
df.head()

In [ ]:
df.groupby(['system', 'environment', 'process'])['instance'].count()

In [ ]:
df.groupby(['process'])['instance'].count()

In [ ]:
G.nodes(data=True)
cytoscape_obj = ipycytoscape.CytoscapeWidget()
cytoscape_obj.graph.add_graph_from_networkx(G, directed=True)
cytoscape_obj.set_style(
    [
        {
            'selector': 'node',
            'style': {
                'font-family': 'helvetica',
                'font-size': 'data(fontsize)',
                'label': 'data(label)',
                'background-color': 'lightblue',
                "text-valign": "center",
                "text-halign": "center",
            }
        },
        {
            "selector": "edge.directed",
            "style": {
                "curve-style": "bezier",
                "target-arrow-shape": "triangle"
            },
        }
    ]
)
cytoscape_obj